#**ANÁLISE EVOLUTIVA DA INTERCONEXÃO DAS LINGUAGENS DE PROGRAMAÇÃO**



In [ ]:
# Bibliotecas Utilizadas Para Execução do Código
import csv
import pandas as pd
from datetime import datetime
import networkx as nx
import matplotlib.pyplot as plt

# **Tratamento da base de dados**

> Primeiramente, foi-se extraído do site www.kaggle.com uma base de dados dos repositórios do GitHub, dos anos de 2008 à 2023. Tais repositórios apresentam diversas características:

* repo_id                     
* owner_username
* repo_name
* description
* created_at
* pushed_at
* size
* stargazers_count
* has_projects
* has_wiki
* has_pages
* forks_count
* open_issues_count
* license
* is_template
* topics
* watching
* contributors_count
* commits_count
* languages
* readme

Para esse trabalho, a coluna *languagens* foi a escolhida como extremamente relevante, tendo em vista que a interconexão das linguagens se dá quando elas são usadas no mesmo repositório em questão.

[Repositories.csv](https://www.kaggle.com/datasets/aditijuneja/github-bipartite-graph-datasetdevelopersrepos)

In [ ]:
#Passo 1: Remover as colunas que não são importantes para esse trabalho
# Carregar o dataframe
df_repositories = pd.read_csv('/content/repositories.csv')

# Remover colunas no dataframe df_repositories
colunas_a_remover_repositories = ['repo_id', 'pushed_at', 'size', 'has_projects', 'has_wiki',
                                  'has_pages', 'license', 'is_template', 'topics', 'watching', 'commits_count', 'readme']
df_repositories = df_repositories.drop(columns=colunas_a_remover_repositories, errors='ignore')

# Salvar o dataframe modificado
df_repositories.to_csv('/content/repositories_etapa_1.csv', index=False)

In [ ]:
#Passo 2: Tratar a data no dataframe repositories.csv
# Carregar o CSV para um DataFrame do Pandas
df_repositories_etapa_1 = pd.read_csv('/content/repositories_etapa_1.csv')

# Função para extrair o ano de uma string de data
def extrair_ano(data):
    data_objeto = datetime.strptime(data, '%Y-%m-%d %H:%M:%S')
    return str(data_objeto.year)

# Aplicar a função à coluna 'created_at'
df_repositories_etapa_1['created_at'] = df_repositories_etapa_1['created_at'].apply(extrair_ano)

# Salvar o DataFrame modificado de volta ao CSV
df_repositories_etapa_1.to_csv('/content/repositories_etapa_2.csv', index=False)

In [ ]:
# Passo 3: Tratar as linguagens vazias no dataframe repositories.csv
# Carregar o dataframe do arquivo CSV
df_repositories_etapa_2 = pd.read_csv('/content/repositories_etapa_2.csv')

# Excluir linhas onde a coluna 'languages' é igual a '[]'
df_repositories_etapa_2 = df_repositories_etapa_2[df_repositories_etapa_2['languages'] != '[]']

# Salvar o dataframe modificado
df_repositories_etapa_2.to_csv('/content/repositories_etapa_3.csv', index=False)

In [ ]:
# Passo 6: Tirar os repositórios que só tem uma linguagem (Interconexção)
# Carregar o dataframe do arquivo CSV
df_repositories = pd.read_csv('/content/repositories_etapa_3.csv')

# Excluir os repositórios que só tem uma linguagem
df_repositories = df_repositories[df_repositories['languages'].apply(lambda x: len(eval(x)) > 1)]

#Salvar o dataframe modificado
df_repositories.to_csv('/content/repositories_final.csv', index=False)

In [ ]:
# Extrair a base de 2010
# Extração das bases de 2010 - 2015 - 2020

# Carregar o dataframe repositories tratado
df_repositories = pd.read_csv('/content/repositories_final.csv', parse_dates=['created_at'])

# Filtrar as linhas onde 'created_at' começa com o ano desejado
df_repositories_2020 = df_repositories[df_repositories['created_at'].dt.year == 2020]

# Salvando o DataFrame resultante em um novo arquivo CSV
df_repositories_2020.to_csv('/content/repositories_2020.csv', index=False, date_format='%Y')

# Construção do **Grafo**

In [ ]:
# Construção do Grafo (2010 - 2015 - 2020)

# Carregando o arquivo CSV
df_repositories_2020 = pd.read_csv('/content/repositories_2020.csv')

# Extraindo a coluna de linguagens do DataFrame
df_repositories_2020_languages = df_repositories_2020['languages']

# Inicializando um grafo não direcionado usando NetworkX
cont = 0
H = nx.Graph()
mapa = dict()

# Iterando sobre as strings na coluna 'languages'
for row in df_repositories_2020_languages:
    languages_list = [lang.strip("'[] ") for lang in row.split(',')]
    aux = languages_list[0]
    cont = 0
    for _ in languages_list:
      aux = languages_list[cont]
      cont += 1
      for lang in languages_list:
        if aux != lang:
          if aux not in mapa:
              mapa[aux] = {lang: 1}
          else:
              if lang in mapa[aux]:
                  mapa[aux][lang] += 1
              else:
                  mapa[aux][lang] = 1

# Adicionando arestas ponderadas ao grafo
for No_1, list_adjacent in mapa.items():
  for No_2, Peso in list_adjacent.items():
    H.add_edge(No_1, No_2, weight = Peso)

# Configurando a visualização do grafo com Kamada-Kawai
fig, ax = plt.subplots(figsize=(25, 20))
edge_map = dict(zip(H.edges(), range(len(H.edges()))))
edge_colors = ["black"] * len(H.edges())
line_width = [1] * len(H.edges())
node_size_factor = 50
node_size = [node_size_factor * H.degree[node] for node in H.nodes()]
nx.draw_kamada_kawai(
    H, with_labels=True, font_weight="bold", edge_color=edge_colors, width=line_width, node_size=node_size
)
plt.show()
print()

# Construção da **Árvore Geradora Máxima**

In [ ]:
# Obtendo a árvore de extensão máxima
maximo = nx.maximum_spanning_tree(H)
new_Graph = nx.Graph()
for edge in maximo.edges(data=True):
  if edge[2]['weight'] > 5:
    new_Graph.add_edge(edge[0], edge[1], weight=edge[2]['weight'])

# Configurando a visualização do grafo com Kamada-Kawai
fig, ax = plt.subplots(figsize=(15, 10))
pos = nx.spring_layout(new_Graph)
nx.draw(new_Graph, pos, with_labels=True, font_weight="bold")
edge_labels = nx.get_edge_attributes(new_Graph, 'weight')
nx.draw_networkx_edge_labels(new_Graph, pos, edge_labels=edge_labels, font_color='red')
plt.show()